## Setup

Set up some basic stuff. We're expecting version 9 of the JSON format at the moment.

In [ ]:
%matplotlib inline

from datetime import datetime

import json
import operator
import numpy as np
import pandas as pd
import matplotlib as mpl

from decimal import Decimal
from pandas.io.json import json_normalize
from functools import reduce

mpl.rcParams['savefig.dpi'] = 1.5 * mpl.rcParams['savefig.dpi']
mpl.style.use('ggplot')

pd.set_option('display.max_columns', 15)
pd.set_option('display.width', 400)

Set up some constants used in the JSON.

In [ ]:
TX_TYPES = {
    'EXPENSE': 1,
    'INCOME': 2,
    'TRANSFER': 3
}

In [ ]:
json_data = json.load(open("./Financius 2016-06-11 180827.json"))

if json_data['version'] != 9:
    raise RuntimeError("Unsupported Financius JSON data version {}".format(json_data['version']))

# Extract all keys that we care about as separate data frames
data = {key: json_normalize(json_data[key]) for key in ['currencies', 'categories', 'tags', 'accounts', 'transactions']}

# Merge categories into transactions since they are n:1
tx = pd.DataFrame.merge(
    data['transactions'],
    data['categories'],
    how='left',
    left_on=['category_id'],
    right_on=['id'],
    suffixes=('_tx', '_cat'))

# Treat the date as ms timestamp and set it as index
tx['date'] = pd.to_datetime(tx['date'], unit='ms')
tx.index = tx['date']

# Keep the original amount
tx['oamount'] = tx['amount']

# Then normalize the 'amount' field by the exchange rate
tx['amount'] = tx.apply(lambda row: row['oamount'] * row['exchange_rate'], axis=1)

# Convert the amount to a float (should really be a decimal, but pandas doesn't like them atm)
tx['amount'] = tx['amount'].apply(lambda x: x / 100)

# Truncate at 2014-01-01
tx = tx.sort_index().truncate(before='2014-01-01')

Manually join the transactions with their tags.

In [ ]:
# Expand the tags into its own dataframe
tag_df = tx['tag_ids'].apply(pd.Series)
tag_df = tag_df.applymap(lambda x: (data['tags'][data['tags']['id'] == x].title).tolist())

# Combine all tag columns into a list
tag_df = tag_df.apply(lambda row: reduce(operator.add, [row[c] for c in tag_df.columns]), axis=1)

# Get the tags back into the transactions df
tx['tags'] = tag_df

# Clean up some of the columns
tx.drop([
        'tag_ids',
        'color',
        'sync_state_tx',
        'sync_state_cat',
        'id_cat',
        'sort_order',
        'transaction_type_cat',
        'exchange_rate',
        'color',
        'model_state_cat',
        'model_state_tx'], axis=1, inplace=True)
tx.head(10)

Split up transactions by their type.

In [ ]:
# group_monthly = lambda x: x.groupby((x.index.month, x.index.year)).aggregate(np.sum)
group_monthly = lambda x: x.groupby(pd.TimeGrouper(freq='M')).aggregate(np.sum)

expenses = tx[tx['transaction_type_tx'] == TX_TYPES['EXPENSE']]
expenses_m = group_monthly(expenses)
incomes = tx[tx['transaction_type_tx'] == TX_TYPES['INCOME']]
incomes_m = group_monthly(incomes)
transfers = tx[tx['transaction_type_tx'] == TX_TYPES['TRANSFER']]
transfers_m = group_monthly(transfers)

## Totals

In [ ]:
total_income = incomes['amount'].sum()
total_expenses = expenses['amount'].sum()

print("Total income: GBP {:.2f}".format(total_income))
print("Total expenses: GBP {:.2f}".format(total_expenses))
print("Total prof: GBP {:.2f}".format(total_income - total_expenses))

## Salary

In [ ]:
salary_line = group_monthly(incomes[incomes['title'] == 'Salary'][['amount']]).rename(columns={'amount': 'Salary'})
tag_filter = lambda name: incomes['tags'].map(lambda tags: name in tags)

p_inc = salary_line.plot()
p_inc.set_ylabel('GBP')

group_monthly(incomes[tag_filter('Freelancing')][['amount']]).rename(columns={'amount': 'Freelancing'}).plot(ax=p_inc)

# df = salary_line.reset_index()
# pd.ols(x=df.date.map(lambda x: x.timestamp()), y=df.Salary)

## Biggest Expenses

In [ ]:
tag_filter = lambda name: expenses['tags'].map(lambda tags: name in tags)
non_rent_expenses = expenses[~(tag_filter('Rent and Mortgage')) & ~(tag_filter('Taxes')) & ~(tag_filter('Insurance')) & (expenses['title'] != 'Other Expense')]
non_rent_expenses.sort_values('amount', ascending=False)[:20][["amount", "title", "note", "tags"]]

## Expenses by Category

In [ ]:
expenses_by_category = expenses.groupby('title')['amount'].sum()
expenses_by_category.sort()
expenses_by_category.plot(kind='barh')

## Income vs Expenses

In [ ]:
p_ie = group_monthly(incomes[["amount"]]).rename(columns={'amount': 'Income'}).plot()
group_monthly(expenses[["amount"]]).rename(columns={'amount': 'Expenses'}).plot(ax=p_ie)
p_ie.set_ylabel("GBP")

In [ ]:
p = mpl.pyplot.subplot()
p.plot(incomes_m['amount'], label="income")
p.plot(expenses_m['amount'], label="expenses")

earnings = incomes_m['amount'] - expenses_m['amount']
p.plot(earnings, label="earnings")

p.set_ylabel('GBP')
p.legend(loc=0)

## Earnings by Month

In [ ]:
earnings_month_index = pd.Series(earnings.index.map(lambda x: ("{}-{:02}".format(x.year, x.month), earnings[x])))
earnings_month_index

# TODO: Turn this into a proper DF

## Cumulative Savings

In [ ]:
print(earnings.sum())
earnings.cumsum().plot()

## Category stats

In [ ]:
expenses[~(tag_filter('Rent and Mortgage')) & (expenses['title'] != 'Other Expense')].groupby('title')['amount'].sum().plot(kind='barh', figsize=(6, 10))

## Coffee

In [ ]:
coffee_exp = group_monthly(expenses[(expenses["title"] == "Food") & (tag_filter("Coffee and Snacks"))][["amount"]])
p = coffee_exp.plot()
p.set_ylabel('GBP')
p.set_xlabel('Month')
p.axhline(coffee_exp.mean()["amount"])

## Cumulative Coffee Expenses

In [ ]:
coffee_exp[['amount']].cumsum().plot()

## Fitness

In [ ]:
fitness_exp = group_monthly(expenses[expenses['title'] == 'Fitness'])
p = fitness_exp[['amount']].plot()
p.set_ylabel('GBP')
p.set_xlabel('Month')
p.axhline(fitness_exp.mean()["amount"])